In [1]:
import torch
import numpy as np
import pandas as pd
import random as rd
import functions as ls
import csv

In [2]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 100, 3
learning_rate = 1e-0

#weight matrix
w1 = torch.randn(N_hid, N_in, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_out, N_hid, device=device, dtype=dtype, requires_grad=True)
#biases
b1 = torch.zeros(N_hid, 1, device=device, dtype=dtype, requires_grad=True)
b2 = torch.zeros(N_out, 1, device=device, dtype=dtype, requires_grad=True)

In [3]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_train_s.csv')
test = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_test_s.csv')
data_np_x = data[['0', '1']].to_numpy()
data_np_y = data[['2']].to_numpy()
data_test = test[['0', '1']].to_numpy()
data_np_y = data_np_y.astype(int)

In [4]:
data_np_x = data_np_x.transpose()
data_np_y = data_np_y.transpose()
data_test = data_test.transpose()
print(data_np_x.shape, data_np_y.shape)

(2, 300) (1, 300)


In [5]:
batchesX = np.asarray(np.array_split(data_np_x, 6, axis=1))
batchesY = np.asarray(np.array_split(data_np_y, 6, axis=1))

In [6]:
Xtest = torch.from_numpy(data_test)

In [7]:
for epoch in range(500):
    b = rd.randint(0, 5)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = (w1.mm(Xtr) + b1).clamp(min=0)
    y = w2.mm(y) + b2
    # SVM loss
    # loss = ls.SVM_loss(y, Ytr)
    # cross entropy loss
    loss = ls.cross_entropy(y, Ytr) + 0.5 * ls.regularization_loss(w1, w2)
    # print(loss.shape, ls.regularization_loss(w1, w2).shape)
    if epoch%100 == 0:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        
        b1 -= learning_rate*b1.grad
        b2 -= learning_rate*b2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()

0 12.522450305247295
100 4.650218043221837
200 5.565405872565498
300 8.033413895670432
400 4.883940908611376


In [10]:
prediction = w2.mm(w1.mm(Xtest).clamp(min=0))
prediction = prediction.T
print(prediction.shape)

torch.Size([300, 3])


In [11]:
pred = open("/home/burrito/Documents/python/pytorch/Python-net/color_classifier/pred_s.csv", 'w', newline='')
writer = csv.writer(pred)
writer.writerow(["color"])
for i in prediction:
    value, index = torch.max(i, 0)
    writer.writerow([index])